In [1]:
import os
# To disable gpu warnings
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

# Pre-process

In [64]:
def determine_label(l):
    return int(l in {'ok', 'normal', 'moved', 'slightly_moved'})

def instance_reader(file_path):
    def _reader():
        eof = False
        with open(file_path) as f:
            while True:
                while True:
                    _tmp = f.readline()
                    if not _tmp:
                        eof = True
                        break
                    elif _tmp != '\n':
                        break

                if eof:
                    break

                label = None
                instance = np.zeros((15, 6))

                # GHD
                label = determine_label(_tmp.strip())
                
                for i in range(15):
                    content = f.readline()
                    timeline = np.asarray(content[:-1].split('\t')[1:]).astype(np.int64)
                    instance[i] = np.asarray(timeline)

                yield instance, label
    return _reader

In [65]:
data_directory = 'data'
X = list()
y = list()
for datafile in os.listdir(data_directory):
    ins_reader = instance_reader(os.path.join(data_directory, datafile))
    for instance, label in tqdm(ins_reader()):
        X.append(instance)
        y.append(label)

X = np.array(X)
y = np.array(y)

88it [00:00, 5176.77it/s]
47it [00:00, 5542.25it/s]
47it [00:00, 5891.58it/s]
117it [00:00, 6173.22it/s]
164it [00:00, 5667.60it/s]


In [66]:
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)
xtrain, xval, ytrain, yval = train_test_split(xtrain, ytrain, test_size=0.125, random_state=42)

## GHD

In [67]:
xtrain[0][0]

array([12.,  8., 94., -9., 35., -1.])

## Elman Neural Network

In [154]:
class ElmanNeuralNetwork(tf.keras.Model):
    def __init__(self, input_dim, hidden_units, feature_size, n_classes):
        super(ElmanNeuralNetwork, self).__init__()
        self.hidden_units = hidden_units
        self.feature_size = feature_size
        self.input_dim = input_dim
        self.n_classes = n_classes

        self.W = self.add_weight(shape=(self.hidden_units, self.feature_size), initializer='random_normal', trainable=True)
        self.U = self.add_weight(shape=(self.hidden_units, self.hidden_units), initializer='random_normal', trainable=True)
        self.b = self.add_weight(shape=(self.hidden_units,), initializer='random_normal', trainable=True)

        self.W_y = self.add_weight(shape=(self.n_classes, self.hidden_units), initializer='random_normal', trainable=True)
        self.b_y = self.add_weight(shape=(self.n_classes,), initializer='random_normal', trainable=True)

    def call(self, x):
        state_t = tf.zeros(self.hidden_units)
        for i in range(self.input_dim):
            state_t = tf.keras.activations.tanh(tf.matmul(self.W, tf.reshape(x[0][i], (self.feature_size, 1))) + tf.matmul(self.U, tf.reshape(state_t, (self.hidden_units,1))) + tf.reshape(self.b, (self.hidden_units, 1)))

        y = tf.keras.activations.tanh(tf.matmul(self.W_y, tf.reshape(state_t, (self.hidden_units,1))) + tf.reshape(self.b_y, (self.n_classes,1)))
        return y

In [155]:
import datetime
elman_nn = ElmanNeuralNetwork(input_dim=15, hidden_units=10, feature_size=6, n_classes=2)

elman_nn.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                loss='binary_crossentropy',
                metrics=['acc'])
elman_nn.fit(
    xtrain,
    ytrain,
    batch_size=1,
    epochs=30,
    validation_data=(xval, yval),
    callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/datetime-elman-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])

Epoch 1/30
323/323 [==============================] - 2s 4ms/step - loss: 2.8160 - acc: 0.6966 - val_loss: 2.6689 - val_acc: 0.7234
Epoch 2/30
323/323 [==============================] - 1s 2ms/step - loss: 2.7236 - acc: 0.7214 - val_loss: 2.6619 - val_acc: 0.7128
Epoch 3/30
323/323 [==============================] - 1s 2ms/step - loss: 2.7106 - acc: 0.7368 - val_loss: 2.6580 - val_acc: 0.7234
Epoch 4/30
323/323 [==============================] - 1s 2ms/step - loss: 2.6988 - acc: 0.7508 - val_loss: 2.6527 - val_acc: 0.7340
Epoch 5/30
323/323 [==============================] - 1s 2ms/step - loss: 2.6929 - acc: 0.7523 - val_loss: 2.6460 - val_acc: 0.7447
Epoch 6/30
323/323 [==============================] - 1s 2ms/step - loss: 2.6917 - acc: 0.7539 - val_loss: 2.6460 - val_acc: 0.7340
Epoch 7/30
323/323 [==============================] - 1s 2ms/step - loss: 2.6852 - acc: 0.7585 - val_loss: 2.6550 - val_acc: 0.7128
Epoch 8/30
323/323 [==============================] - 1s 3ms/step - loss: 2.

In [126]:
%load_ext tensorboard

In [132]:
# load tensorboard
%tensorboard --logdir=logs/

Reusing TensorBoard on port 6006 (pid 11948), started 0:00:46 ago. (Use '!kill 11948' to kill it.)

## Jordan

In [152]:
class JordanNeuralNetwork(tf.keras.Model):
    def __init__(self, input_dim, hidden_units, feature_size, n_classes):
        super(JordanNeuralNetwork, self).__init__()
        self.hidden_units = hidden_units
        self.feature_size = feature_size
        self.input_dim = input_dim
        self.n_classes = n_classes

        self.W = self.add_weight(shape=(self.hidden_units, self.feature_size), initializer='random_normal', trainable=True)
        self.U = self.add_weight(shape=(self.hidden_units, self.hidden_units), initializer='random_normal', trainable=True)
        self.b = self.add_weight(shape=(self.hidden_units,), initializer='random_normal', trainable=True)

        self.W_y_intermediate = self.add_weight(shape=(self.hidden_units, self.hidden_units), initializer='random_normal', trainable=True)
        self.b_y_intermediate = self.add_weight(shape=(self.hidden_units,), initializer='random_normal', trainable=True)

        self.W_y = self.add_weight(shape=(self.n_classes, self.hidden_units), initializer='random_normal', trainable=True)
        self.b_y = self.add_weight(shape=(self.n_classes,), initializer='random_normal', trainable=True)

    def call(self, x):
        state_t = tf.zeros(self.hidden_units)
        for i in range(self.input_dim):
            hidden = tf.keras.activations.tanh(tf.matmul(self.W, tf.reshape(x[0][i], (self.feature_size, 1))) + tf.matmul(self.U, tf.reshape(state_t, (self.hidden_units,1))) + tf.reshape(self.b, (self.hidden_units, 1)))
            y_intermediate = tf.keras.activations.tanh(tf.matmul(self.W_y_intermediate, tf.reshape(hidden, (self.hidden_units,1))) + tf.reshape(self.b_y_intermediate, (self.hidden_units,1)))
            state_t = y_intermediate
        y = tf.keras.activations.tanh(tf.matmul(self.W_y, tf.reshape(state_t, (self.hidden_units,1))) + tf.reshape(self.b_y, (self.n_classes,1)))

        return y

In [153]:

jordan_nn = JordanNeuralNetwork(input_dim=15, hidden_units=10, feature_size=6, n_classes=2)

jordan_nn.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                loss='binary_crossentropy',
                metrics=['acc'])
jordan_nn.fit(
    xtrain,
    ytrain,
    batch_size=1,
    epochs=30,
    validation_data=(xval, yval),
    callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/datetime-jordan-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])

Epoch 1/30
323/323 [==============================] - 3s 5ms/step - loss: 0.5916 - acc: 0.6672 - val_loss: 0.4042 - val_acc: 0.7766
Epoch 2/30
323/323 [==============================] - 1s 3ms/step - loss: 0.3818 - acc: 0.7833 - val_loss: 0.3907 - val_acc: 0.8085
Epoch 3/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3659 - acc: 0.7910 - val_loss: 0.4325 - val_acc: 0.7447
Epoch 4/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3528 - acc: 0.8111 - val_loss: 0.3962 - val_acc: 0.7021
Epoch 5/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3385 - acc: 0.8111 - val_loss: 0.3859 - val_acc: 0.7872
Epoch 6/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3478 - acc: 0.8034 - val_loss: 0.3699 - val_acc: 0.7872
Epoch 7/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3276 - acc: 0.8282 - val_loss: 0.3734 - val_acc: 0.7447
Epoch 8/30
323/323 [==============================] - 1s 2ms/step - loss: 0.

## Ensemble

In [187]:
elman_nn1 = ElmanNeuralNetwork(input_dim=15, hidden_units=10, feature_size=6, n_classes=2)

elman_nn1.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                loss='binary_crossentropy',
                metrics=['acc'])
elman_nn1.fit(
    xtrain,
    ytrain,
    batch_size=1,
    epochs=30,
    validation_data=(xval, yval),
    callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/datetime-elman1-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])


Epoch 1/30
323/323 [==============================] - 3s 5ms/step - loss: 0.7514 - acc: 0.6827 - val_loss: 0.4031 - val_acc: 0.7128
Epoch 2/30
323/323 [==============================] - 1s 3ms/step - loss: 0.4000 - acc: 0.7260 - val_loss: 0.3797 - val_acc: 0.7553
Epoch 3/30
323/323 [==============================] - 1s 3ms/step - loss: 0.3702 - acc: 0.7864 - val_loss: 0.3920 - val_acc: 0.7447
Epoch 4/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3517 - acc: 0.8142 - val_loss: 0.3884 - val_acc: 0.7234
Epoch 5/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3414 - acc: 0.8328 - val_loss: 0.3839 - val_acc: 0.7553
Epoch 6/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3275 - acc: 0.8328 - val_loss: 0.3780 - val_acc: 0.7340
Epoch 7/30
323/323 [==============================] - 1s 3ms/step - loss: 0.3228 - acc: 0.8313 - val_loss: 0.3866 - val_acc: 0.7128
Epoch 8/30
323/323 [==============================] - 1s 2ms/step - loss: 0.

In [200]:
elman_nn2 = ElmanNeuralNetwork(input_dim=15, hidden_units=20, feature_size=6, n_classes=1)
elman_nn2.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                loss='binary_crossentropy',
                metrics=['acc'])
elman_nn2.fit(
    xtrain,
    ytrain,
    batch_size=1,
    epochs=30,
    validation_data=(xval, yval),
    callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/datetime-elman2-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])




Epoch 1/30
323/323 [==============================] - 3s 5ms/step - loss: 0.9278 - acc: 0.6904 - val_loss: 0.3818 - val_acc: 0.8085
Epoch 2/30
323/323 [==============================] - 1s 4ms/step - loss: 0.3574 - acc: 0.7926 - val_loss: 0.3897 - val_acc: 0.8085
Epoch 3/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3387 - acc: 0.8266 - val_loss: 0.3593 - val_acc: 0.8085
Epoch 4/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3138 - acc: 0.8328 - val_loss: 0.3578 - val_acc: 0.7660
Epoch 5/30
323/323 [==============================] - 1s 2ms/step - loss: 0.2963 - acc: 0.8297 - val_loss: 0.3595 - val_acc: 0.7447
Epoch 6/30
323/323 [==============================] - 1s 2ms/step - loss: 0.2889 - acc: 0.8545 - val_loss: 0.4593 - val_acc: 0.7872
Epoch 7/30
323/323 [==============================] - 1s 2ms/step - loss: 0.2829 - acc: 0.8359 - val_loss: 0.3575 - val_acc: 0.8085
Epoch 8/30
323/323 [==============================] - 1s 2ms/step - loss: 0.

In [204]:
id=6
print(ytrain[id])

1


In [205]:
elman_nn1(xtrain[np.newaxis,id])

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.95896643],
       [0.9612518 ]], dtype=float32)>

In [206]:
elman_nn2(xtrain[np.newaxis,id])

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.9331546]], dtype=float32)>

## loading data

In [254]:
segments = []
labels = []
label_index = []
i = 0
with open('data/lp1.data', 'r') as f:
    prev_line_hoo = False
    for line in f.readlines()[:40]:
        if line.startswith("\n"):
            if prev_line_hoo:
                print("NNNNNN", end="")
                # segments.append("HOOOOOO")
            prev_line_hoo = True
        elif line.startswith("\t"):
            print("TAAAAB", end="")
            segments.append(list(map(int, line.split())))
        elif line == "\n":
            print("FFFFFF")
            
        else:
            labels.append(line)
            label_index.append(i)
        print(line, end="")
        i += 1

normal
TAAAAB	-1	-1	63	-3	-1	0
TAAAAB	0	0	62	-3	-1	0
TAAAAB	-1	-1	61	-3	0	0
TAAAAB	-1	-1	63	-2	-1	0
TAAAAB	-1	-1	63	-3	-1	0
TAAAAB	-1	-1	63	-3	-1	0
TAAAAB	-1	-1	63	-3	0	0
TAAAAB	-1	-1	63	-3	-1	0
TAAAAB	-1	-1	63	-3	-1	0
TAAAAB	-1	-1	61	-3	0	0
TAAAAB	-1	-1	61	-3	0	0
TAAAAB	-1	-1	64	-3	-1	0
TAAAAB	-1	-1	64	-3	-1	0
TAAAAB	-1	-1	60	-3	0	0
TAAAAB	-1	0	64	-2	-1	0

NNNNNN
normal
TAAAAB	-1	-1	63	-2	-1	0
TAAAAB	-1	-1	63	-3	-1	0
TAAAAB	-1	-1	61	-3	0	0
TAAAAB	0	-4	63	1	0	0
TAAAAB	0	-1	59	-2	0	-1
TAAAAB	-3	3	57	-8	-3	-1
TAAAAB	-1	3	70	-10	-2	-1
TAAAAB	0	-3	61	0	0	0
TAAAAB	0	-2	53	-1	-2	0
TAAAAB	0	-3	66	1	4	0
TAAAAB	-3	3	58	-10	-5	0
TAAAAB	-1	-1	66	-4	-2	0
TAAAAB	-1	-2	67	-3	-1	0
TAAAAB	0	1	66	-6	-3	-1
TAAAAB	-1	-1	59	-3	-4	0
NNNNNN
NNNNNN
normal
TAAAAB	-1	0	57	-5	-3	0
TAAAAB	0	-3	63	-1	0	0
TAAAAB	-1	1	51	-4	-1	-1


In [262]:
segments = []
labels = []
label_index = []
i = 0
with open('data/lp1.data', 'r') as f:
    prev_line_hoo = False
    for line in f.readlines()[:100]:
        if line.startswith("\n"):
            pass
        elif line.startswith("\t"):
            segments.append(list(map(int, line.split())))
        elif line == "\n":
            pass
        else:
            labels.append(line)
            label_index.append(i)
        i += 1

In [268]:
label_index

[0, 18, 36, 54, 72, 90]

In [263]:
for i in range(len(segments)):
    if i in label_index:
        print()
        print(labels[label_index.index(i)][:-1])
        print(" ".join(list(map(str, segments[i]))))
    else:
        print(" ".join(list(map(str, segments[i]))))


normal
-1 -1 63 -3 -1 0
0 0 62 -3 -1 0
-1 -1 61 -3 0 0
-1 -1 63 -2 -1 0
-1 -1 63 -3 -1 0
-1 -1 63 -3 -1 0
-1 -1 63 -3 0 0
-1 -1 63 -3 -1 0
-1 -1 63 -3 -1 0
-1 -1 61 -3 0 0
-1 -1 61 -3 0 0
-1 -1 64 -3 -1 0
-1 -1 64 -3 -1 0
-1 -1 60 -3 0 0
-1 0 64 -2 -1 0
-1 -1 63 -2 -1 0
-1 -1 63 -3 -1 0
-1 -1 61 -3 0 0

normal
0 -4 63 1 0 0
0 -1 59 -2 0 -1
-3 3 57 -8 -3 -1
-1 3 70 -10 -2 -1
0 -3 61 0 0 0
0 -2 53 -1 -2 0
0 -3 66 1 4 0
-3 3 58 -10 -5 0
-1 -1 66 -4 -2 0
-1 -2 67 -3 -1 0
0 1 66 -6 -3 -1
-1 -1 59 -3 -4 0
-1 0 57 -5 -3 0
0 -3 63 -1 0 0
-1 1 51 -4 -1 -1
-1 -2 68 -2 -2 0
-1 -1 65 -6 1 0
0 0 61 -5 -2 0

normal
-1 1 61 -6 0 -1
0 -3 57 3 -4 0
-1 -1 59 -4 -4 0
1 -3 65 -1 1 0
-1 2 64 -7 -2 0
-1 1 66 -7 -3 -1
-1 0 61 -5 -5 0
-1 0 65 -6 -2 -1
-1 0 54 -4 -3 0
0 -1 59 -2 -1 -1
0 -3 61 -1 2 0
-2 1 56 -6 -3 0
1 -3 64 -1 4 0
-1 1 62 -7 1 -1
-1 0 60 -9 -5 -1
1 1 56 -5 0 0
1 -1 66 -4 2 1
-2 5 64 -15 -2 0

normal
-1 2 58 -8 -4 0
0 1 70 -9 -2 -1
-1 1 64 -8 -6 -1
0 -1 67 -6 0 -1
0 -2 63 -4 0 0
-1 1 63 -8 -2 0